# LSTM in Pytorch

In [75]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

## Basic LSTM in Pytorch with random numbers

In [76]:
#input
x = torch.tensor([[1,2, 12,34, 56,78, 90,80],
                 [12,45, 99,67, 6,23, 77,82],
                 [3,24, 6,99, 12,56, 21,22]])

#### using two different models

In [77]:
model1 = nn.Embedding(100, 7, padding_idx=0)
model2 = nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True)

In [78]:
out1 = model1(x)
out2 = model2(out1)

In [79]:
print(out1.shape)
print(out1)

torch.Size([3, 8, 7])
tensor([[[ 1.4293e+00,  6.0164e-01,  7.6816e-01, -1.0571e+00,  2.7667e-01,
           3.5624e-01, -1.0728e+00],
         [-1.0835e-01, -7.5990e-01,  9.6901e-01,  1.1652e+00, -4.7132e-01,
          -3.4492e-01,  8.8614e-02],
         [-1.0716e+00, -4.1759e-02, -1.4939e-01, -1.8338e+00,  5.1733e-01,
          -1.6915e+00, -9.4595e-01],
         [ 1.5975e-01,  1.6867e+00,  2.6485e-01, -6.9314e-02, -6.7790e-02,
           1.9867e-01,  5.2259e-01],
         [-3.8025e-01, -7.0423e-01, -1.1162e-01, -3.7739e-01,  7.9428e-01,
          -1.1129e+00, -1.5543e+00],
         [ 7.1977e-01, -1.2741e+00, -7.6088e-01,  5.4890e-01, -5.7841e-01,
          -1.2679e+00,  1.0528e+00],
         [ 2.3582e-03, -2.1939e-02,  1.5134e+00, -1.0499e+00,  1.9289e-01,
           9.7143e-01, -1.8020e+00],
         [ 1.0041e-01, -5.1223e-01,  1.4013e+00,  1.0783e+00, -1.1330e+00,
          -4.1461e-01,  9.4424e-01]],

        [[-1.0716e+00, -4.1759e-02, -1.4939e-01, -1.8338e+00,  5.1733e-01,
     

In [80]:
out, (ht, ct) = model2(out1)
print(ht)

tensor([[[-0.1052, -0.1486, -0.0142],
         [ 0.2526, -0.0047, -0.0837],
         [-0.0167, -0.0155,  0.0059]]], grad_fn=<StackBackward0>)


#### using nn.sequential

In [81]:
model3 = nn.Sequential(nn.Embedding(100, 7, padding_idx=0),
                        nn.LSTM(input_size=7, hidden_size=3, num_layers=1, batch_first=True))

In [82]:
out, (ht, ct) = model3(x)
print(out)

tensor([[[ 0.0139,  0.0528, -0.2210],
         [-0.0319, -0.2507, -0.2080],
         [-0.1176, -0.3840, -0.0276],
         [-0.2235, -0.5089,  0.1662],
         [-0.0465, -0.1223,  0.0292],
         [-0.1320,  0.1203, -0.0972],
         [-0.1008, -0.0468, -0.4051],
         [-0.1746, -0.0149, -0.3570]],

        [[-0.1021, -0.3284,  0.0188],
         [-0.0319, -0.0117,  0.0463],
         [-0.0170,  0.0453,  0.1699],
         [-0.0618, -0.2194, -0.2132],
         [-0.0929, -0.0073, -0.4266],
         [-0.2656, -0.2856, -0.6883],
         [-0.1599, -0.0749, -0.2164],
         [-0.1523, -0.3989, -0.1668]],

        [[-0.0298, -0.1546, -0.1625],
         [ 0.0489,  0.0818, -0.1257],
         [-0.0277,  0.0745, -0.4547],
         [-0.0368,  0.1189, -0.2437],
         [-0.1705, -0.3224, -0.1494],
         [-0.0340, -0.1091, -0.1732],
         [-0.1391, -0.2210, -0.1449],
         [-0.2498, -0.1910, -0.1020]]], grad_fn=<TransposeBackward0>)


## Multiclass Text Classification

We are going to predict item ratings based on customer reviews bsed on this dataset from Kaggle:
https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

In [83]:
#loading the data
reviews = pd.read_csv("reviews.csv")
print(reviews.shape)
reviews.head()

(23486, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [84]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']
reviews

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,review
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and com...
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happen...
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Some major design flaws I had such high hopes ...
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"My favorite buy! I love, love, love this jumps..."
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Flattering shirt This shirt is very flattering...
...,...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,Great dress for many occasions I was very happ...
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits,Wish it was made of cotton It reminds me of ma...
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses,"Cute, but see through This fit well, but the t..."
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses,"Very cute dress, perfect for summer parties an..."


In [85]:
#keeping only relevant columns and calculating sentence lengths
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews['review_length'] = reviews['review'].apply(lambda x: len(x.split()))
reviews.head()

C:\Users\Jose\AppData\Local\Temp\ipykernel_22552\2008228755.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['review_length'] = reviews['review'].apply(lambda x: len(x.split()))


,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,4,8
1,Love this dress! it's sooo pretty. i happen...,5,62
2,Some major design flaws I had such high hopes ...,3,102
3,"My favorite buy! I love, love, love this jumps...",5,25
4,Flattering shirt This shirt is very flattering...,5,38


In [86]:
#changing ratings to 0-numbering
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['rating'] = reviews['rating'].apply(lambda x: zero_numbering[x])
reviews

C:\Users\Jose\AppData\Local\Temp\ipykernel_22552\2365825949.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['rating'] = reviews['rating'].apply(lambda x: zero_numbering[x])


,review,rating,review_length
0,Absolutely wonderful - silky and sexy and com...,3,8
1,Love this dress! it's sooo pretty. i happen...,4,62
2,Some major design flaws I had such high hopes ...,2,102
3,"My favorite buy! I love, love, love this jumps...",4,25
4,Flattering shirt This shirt is very flattering...,4,38
...,...,...,...
23481,Great dress for many occasions I was very happ...,4,33
23482,Wish it was made of cotton It reminds me of ma...,2,44
23483,"Cute, but see through This fit well, but the t...",2,46
23484,"Very cute dress, perfect for summer parties an...",2,95


In [87]:
#mean sentence length
np.mean(reviews['review_length'])

60.832921740611425

In [88]:
#tokenization
# python -m spacy download en
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [89]:
#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['review']))

In [90]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 14138
num_words after: 8263


In [91]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [92]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [93]:
reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
reviews.head()

C:\Users\Jose\AppData\Local\Temp\ipykernel_22552\3512281136.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
C:\Users\Jose\AppData\Local\Temp\ipykernel_22552\3512281136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['encoded'] = reviews['review'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))


,review,rating,review_length,encoded
0,Absolutely wonderful - silky and sexy and com...,3,8,"[[2, 3, 4, 5, 6, 7, 8, 7, 9, 0, 0, 0, 0, 0, 0,..."
1,Love this dress! it's sooo pretty. i happen...,4,62,"[[2, 10, 11, 12, 5, 13, 14, 15, 16, 5, 17, 18,..."
2,Some major design flaws I had such high hopes ...,2,102,"[[54, 55, 56, 57, 17, 58, 59, 60, 61, 62, 11, ..."
3,"My favorite buy! I love, love, love this jumps...",4,25,"[[68, 109, 110, 2, 17, 10, 2, 10, 2, 10, 11, 1..."
4,Flattering shirt This shirt is very flattering...,4,38,"[[122, 123, 11, 123, 52, 92, 122, 19, 124, 125..."


In [94]:
#check how balanced the dataset is
Counter(reviews['rating'])

Counter({3: 5077, 4: 13131, 2: 2871, 1: 1565, 0: 842})

In [95]:
X = list(reviews['encoded'])
y = list(reviews['rating'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

#### Pytorch Dataset

In [22]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [23]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [24]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [25]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

### LSTM with fixed length input

In [26]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [27]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [28]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 1.239, val loss 1.218, val accuracy 0.556, and val rmse 1.355
train loss 1.188, val loss 1.203, val accuracy 0.554, and val rmse 1.348
train loss 1.112, val loss 1.165, val accuracy 0.574, and val rmse 1.189
train loss 1.072, val loss 1.161, val accuracy 0.517, and val rmse 1.145
train loss 1.042, val loss 1.117, val accuracy 0.563, and val rmse 1.279
train loss 0.981, val loss 1.113, val accuracy 0.572, and val rmse 1.216


In [29]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.936, val loss 1.065, val accuracy 0.576, and val rmse 1.177
train loss 0.846, val loss 1.009, val accuracy 0.603, and val rmse 0.937
train loss 0.848, val loss 1.009, val accuracy 0.606, and val rmse 0.912
train loss 0.784, val loss 0.994, val accuracy 0.604, and val rmse 0.894
train loss 0.741, val loss 0.984, val accuracy 0.617, and val rmse 0.870
train loss 0.702, val loss 0.999, val accuracy 0.623, and val rmse 0.863


In [30]:
train_model(model_fixed, epochs=30, lr=0.01)

train loss 0.695, val loss 1.010, val accuracy 0.619, and val rmse 0.869
train loss 0.630, val loss 0.992, val accuracy 0.625, and val rmse 0.836
train loss 0.583, val loss 1.020, val accuracy 0.632, and val rmse 0.823
train loss 0.545, val loss 1.052, val accuracy 0.635, and val rmse 0.823
train loss 0.502, val loss 1.088, val accuracy 0.634, and val rmse 0.827
train loss 0.469, val loss 1.145, val accuracy 0.639, and val rmse 0.817


### LSTM with variable length input

In [31]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [32]:
model = LSTM_variable_input(vocab_size, 50, 50)

In [33]:
train_model(model, epochs=30, lr=0.1)

train loss 1.328, val loss 1.250, val accuracy 0.515, and val rmse 1.312
train loss 1.031, val loss 1.063, val accuracy 0.577, and val rmse 1.017
train loss 0.904, val loss 0.995, val accuracy 0.603, and val rmse 0.941
train loss 0.849, val loss 1.000, val accuracy 0.599, and val rmse 0.940
train loss 0.845, val loss 1.009, val accuracy 0.598, and val rmse 0.921
train loss 0.834, val loss 1.005, val accuracy 0.593, and val rmse 0.902


In [34]:
train_model(model, epochs=30, lr=0.05)

train loss 0.828, val loss 1.000, val accuracy 0.599, and val rmse 0.920
train loss 0.790, val loss 0.989, val accuracy 0.605, and val rmse 0.894
train loss 0.775, val loss 0.992, val accuracy 0.614, and val rmse 0.884
train loss 0.755, val loss 0.994, val accuracy 0.597, and val rmse 0.883
train loss 0.738, val loss 0.987, val accuracy 0.608, and val rmse 0.872
train loss 0.741, val loss 1.005, val accuracy 0.611, and val rmse 0.888


In [35]:
train_model(model, epochs=30, lr=0.05)

train loss 0.758, val loss 1.028, val accuracy 0.616, and val rmse 0.884
train loss 0.725, val loss 0.994, val accuracy 0.621, and val rmse 0.877
train loss 0.715, val loss 0.999, val accuracy 0.607, and val rmse 0.881
train loss 0.707, val loss 1.008, val accuracy 0.608, and val rmse 0.879
train loss 0.698, val loss 1.018, val accuracy 0.615, and val rmse 0.890
train loss 0.686, val loss 1.017, val accuracy 0.603, and val rmse 0.893


### LSTM with pretrained Glove word embeddings

Download weights from : https://nlp.stanford.edu/projects/glove/

In [36]:
def load_glove_vectors(glove_file="./data/glove.6B/glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [37]:
def get_emb_matrix(pretrained, word_counts, emb_size = 50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [38]:
word_vecs = load_glove_vectors()
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [39]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [40]:
model = LSTM_glove_vecs(vocab_size, 50, 50, pretrained_weights)

In [41]:
train_model(model, epochs=30, lr=0.1)

train loss 1.281, val loss 1.255, val accuracy 0.556, and val rmse 1.355
train loss 1.210, val loss 1.207, val accuracy 0.556, and val rmse 1.354
train loss 1.206, val loss 1.204, val accuracy 0.556, and val rmse 1.354
train loss 1.201, val loss 1.202, val accuracy 0.556, and val rmse 1.354
train loss 1.173, val loss 1.168, val accuracy 0.557, and val rmse 1.352
train loss 1.131, val loss 1.122, val accuracy 0.562, and val rmse 1.249


In [42]:
train_model(model, epochs=30, lr=0.05)

train loss 1.112, val loss 1.113, val accuracy 0.556, and val rmse 1.349
train loss 1.061, val loss 1.051, val accuracy 0.570, and val rmse 1.109
train loss 1.014, val loss 1.014, val accuracy 0.582, and val rmse 1.058
train loss 0.979, val loss 0.990, val accuracy 0.599, and val rmse 0.995
train loss 0.948, val loss 0.961, val accuracy 0.610, and val rmse 0.950
train loss 0.923, val loss 0.952, val accuracy 0.612, and val rmse 0.935


In [43]:
train_model(model, epochs=30, lr=0.05)

train loss 1.189, val loss 1.014, val accuracy 0.586, and val rmse 1.033
train loss 0.946, val loss 0.964, val accuracy 0.606, and val rmse 0.950
train loss 0.912, val loss 0.951, val accuracy 0.612, and val rmse 0.941
train loss 0.895, val loss 0.949, val accuracy 0.615, and val rmse 0.913
train loss 0.886, val loss 0.947, val accuracy 0.617, and val rmse 0.901
train loss 0.872, val loss 0.938, val accuracy 0.621, and val rmse 0.890


## Predicting ratings using regression instead of classification

In [44]:
def train_model_regr(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.float()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.mse_loss(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss = validation_metrics_regr(model, val_dl)
        if i % 5 == 1:
            print("train mse %.3f val rmse %.3f" % (sum_loss/total, val_loss))

def validation_metrics_regr (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.float()
        y_hat = model(x, l)
        loss = np.sqrt(F.mse_loss(y_hat, y.unsqueeze(-1)).item())
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total

In [45]:
class LSTM_regr(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [46]:
model =  LSTM_regr(vocab_size, 50, 50)

In [47]:
train_model_regr(model, epochs=30, lr=0.05)

train mse 1.663 val rmse 1.313
train mse 1.215 val rmse 1.125
train mse 1.151 val rmse 1.109
train mse 1.114 val rmse 1.115
train mse 1.082 val rmse 1.121
train mse 1.043 val rmse 1.116


In [48]:
train_model_regr(model, epochs=30, lr=0.05)

train mse 1.214 val rmse 1.193
train mse 0.884 val rmse 1.032
train mse 0.631 val rmse 0.903
train mse 0.483 val rmse 0.837
train mse 0.416 val rmse 0.806
train mse 0.363 val rmse 0.799


In [ ]:
jovian.commit("lstm multiclass text classification, regression")

[jovian] Saving notebook..


<IPython.core.display.Javascript object>